In [ ]:
import torch

# Create a tensor
x = torch.rand(5, 3)
print(x)
torch.cuda.is_available()

print(type(x))

In [1]:
import asyncio
import websockets
import nest_asyncio

nest_asyncio.apply()

async def echo(websocket):
    async for message in websocket:
        await websocket.send(message)
        print('\r' + message, end='')

async def main():
    async with websockets.serve(echo, "localhost", 8765):
        await asyncio.Future()  # run forever

asyncio.run(main())